In [ ]:
# %pip install python-dotenv pandas pinecone langchain langchain-openai langchain-pinecone scikit-learn matplotlib


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # 'gpt-4o-mini'
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")  # 'ir'
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))

# 압축 인덱스 이름
COMPRESSED_INDEX_NAME = f"{PINECONE_INDEX_NAME}-compressed"

print(f"환경 변수 로딩 완료 : {PINECONE_INDEX_NAME}, {PINECONE_INDEX_REGION}, {PINECONE_INDEX_CLOUD}")

환경 변수 로딩 완료 : ir-embeddings, us-east-1, aws


In [2]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Pinecone 클라이언트 연결
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)
# Dense 벡터 스토어 설정
vector_store = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embedding_model)

print("Pinecone 설정 완료")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pinecone 설정 완료


In [3]:
import pandas as pd

# 멀티홉 질의 파일 queries_multihop.csv 로드
queries_multihop_df = pd.read_csv("../../datas/queries_multihop.csv")

print("멀티홉 질의 데이터 로드 완료")
print(f"멀티홉 질의 수: {len(queries_multihop_df)}")

멀티홉 질의 데이터 로드 완료
멀티홉 질의 수: 30


In [4]:
import time

def retrieve_1hop(query: str, k: int = 5):
    start = time.time()
    docs_and_scores = vector_store.similarity_search_with_score(query, k=k)
    retrieved_ids = [doc.metadata['doc_id'] for doc, _ in docs_and_scores]
    latency = time.time() - start
    return retrieved_ids, latency

print("1-hop 검색 함수 정의 완료")

1-hop 검색 함수 정의 완료


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Follow-up Query 생성 체인 설정 (LCEL 스타일)
followup_prompt = PromptTemplate(
    input_variables=['original_query', 'hop1_docs_contents'],
    template=(
        "사용자의 복합 질문:\n"
        "{original_query}\n\n"
        "1-hop 검색 결과 문서의 요약/본문:\n"
        "{hop1_docs_contents}\n\n"
        "위 정보를 바탕으로, 복합 질문 해결을 위한 2-hop 서브쿼리를 한 문장으로 작성하세요."
    )
)
output_parser = StrOutputParser()

chat_model = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0
)

followup_chain = followup_prompt | chat_model | output_parser
print("Follow-up Query 생성 체인 설정 완료")

Follow-up Query 생성 체인 설정 완료


In [6]:
def get_docs_content_by_ids(doc_ids: list[str]) -> str:
    contents = []
    for did in doc_ids:
        fetch_response = vector_store.index.fetch(ids=[did])
        vector_data = fetch_response.vectors.get(did)
        if vector_data and 'metadata' in vector_data:
            contents.append(vector_data['metadata']['text'])
    return "\n".join(contents)

print("문서 메타에서 텍스트 결합 함수 정의 완료")

문서 메타에서 텍스트 결합 함수 정의 완료


In [7]:
def precision_at_k(predicted: list[str], relevant: list[str], k: int = 5) -> float:
    hits = sum(1 for doc in predicted[:k] if doc in relevant)
    return hits / k if k > 0 else 0.0

def parse_relevant_ids(relevant_str: str) -> list[str]:
    return [pair.split('=')[0] for pair in relevant_str.split(';') if pair]

print("평가 함수 정의 완료")

평가 함수 정의 완료


In [8]:
results = []
for idx, row in queries_multihop_df.iterrows():
    qid = row['query_id']
    orig_query = row['query_text']
    gt_hop1 = parse_relevant_ids(row['hop1_relevant_ids'])
    gt_hop2 = parse_relevant_ids(row['hop2_relevant_ids'])
    hop1_retrieved, t1 = retrieve_1hop(orig_query, k=5)
    hop1_text = get_docs_content_by_ids(hop1_retrieved)
    sub_query = followup_chain.invoke({'original_query': orig_query, 'hop1_docs_contents': hop1_text})
    hop2_retrieved, t2 = retrieve_1hop(sub_query, k=5)
    p1 = precision_at_k(hop1_retrieved, gt_hop1)
    p2 = precision_at_k(hop2_retrieved, gt_hop2)
    results.append({'query_id': qid, 'hop1_time': t1, 'hop2_time': t2,
                    'original_query': orig_query, 'sub_query': sub_query,
                    'retrieved_hop1': hop1_retrieved, 'retrieved_hop2': hop2_retrieved,
                    'P@5_hop1': p1, 'P@5_hop2': p2})
results_df = pd.DataFrame(results)
print('멀티홉 실험 완료')
results_df.head()

멀티홉 실험 완료


,query_id,hop1_time,hop2_time,original_query,sub_query,retrieved_hop1,retrieved_hop2,P@5_hop1,P@5_hop2
0,Q01,1.994064,0.954649,저자 김민수이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"""김민수가 쓴 문서와 동일한 카테고리에 속하는 다른 저자의 문서를 찾아주세요.""","[D29, D29, D28, D28, D17]","[D29, D29, D28, D28, D30]",0.0,0.0
1,Q02,0.629838,0.848791,저자 이영희이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"""이영희 저자의 문서와 동일한 카테고리에 속하는 다른 저자의 문서를 찾아주세요.""","[D17, D17, D29, D29, D30]","[D17, D17, D30, D30, D29]",0.0,0.0
2,Q03,0.742967,0.794559,저자 박지훈이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"""박지훈이 쓴 문서의 카테고리를 기반으로 해당 카테고리에 속하는 다른 저자의 문서를...","[D30, D30, D29, D29, D28]","[D30, D30, D29, D29, D17]",0.0,0.2
3,Q04,0.624264,1.809470,저자 최수정이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"""최수정이 쓴 문서의 카테고리를 기반으로 해당 카테고리에 속하는 다른 저자의 문서를...","[D29, D29, D28, D28, D30]","[D29, D29, D30, D30, D28]",0.0,0.0
4,Q05,0.732533,0.722740,저자 정우성이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"""정우성이 쓴 문서와 동일한 카테고리에 속하는 다른 저자의 문서를 찾아 요약해 주세요.""","[D29, D29, D28, D28, D30]","[D29, D29, D28, D28, D30]",0.0,0.0


In [11]:
results_df.sort_values(by=['P@5_hop1', 'P@5_hop2'], ascending=[False, True])[:10]


,query_id,hop1_time,hop2_time,original_query,sub_query,retrieved_hop1,retrieved_hop2,P@5_hop1,P@5_hop2
18,Q19,0.544515,0.532422,카테고리 기술에 속한 문서를 쓴 저자의 다른 문서를 찾아줘,"""카테고리 기술에 속한 문서를 쓴 저자의 다른 문서를 찾기 위해, 해당 저자가 작성...","[D29, D29, D19, D19, D28]","[D29, D29, D30, D30, D28]",1.0,0.0
24,Q25,0.874834,0.621930,카테고리가 기술인 문서 중 카테고리가 윤리인 문서를 찾아줘,"""기술 카테고리에 속하는 문서 중에서 윤리와 관련된 내용을 포함한 문서를 찾아주세요.""","[D29, D29, D30, D30, D28]","[D29, D29, D25, D25, D30]",1.0,0.0
12,Q13,0.711849,0.524887,카테고리 검색에 속한 문서를 쓴 저자의 다른 문서를 찾아줘,"""카테고리 검색에 속한 문서의 저자가 쓴 다른 문서를 찾기 위해 해당 저자의 이름을...","[D29, D29, D30, D30, D28]","[D29, D29, D30, D30, D28]",1.0,0.2
10,Q11,0.805258,0.744178,카테고리 AI에 속한 문서를 쓴 저자의 다른 문서를 찾아줘,"""AI 카테고리에 속한 특정 저자의 다른 문서들을 찾기 위해 해당 저자의 이름을 기...","[D29, D29, D30, D30, D25]","[D30, D30, D29, D29, D25]",0.8,0.0
22,Q23,0.504295,0.476066,카테고리가 역사인 문서 중 카테고리가 군사인 문서를 찾아줘,"""역사 카테고리에 속하는 문서 중에서 군사 카테고리에 해당하는 문서를 찾는 서브쿼리...","[D5, D5, D29, D29, D15]","[D29, D29, D30, D30, D28]",0.6,0.0
8,Q09,0.606799,0.620142,저자 신동엽이 쓴 문서의 카테고리와 동일한 다른 문서를 찾아줘,"신동엽이 쓴 문서의 카테고리와 동일한 다른 저자의 문서를 찾기 위해, ""신동엽의 문...","[D28, D28, D29, D29, D17]","[D29, D29, D28, D28, D30]",0.4,0.0
21,Q22,0.474119,0.432354,카테고리가 역사인 문서 중 카테고리가 교육인 문서를 찾아줘,"""역사 카테고리에 속하는 문서에서 교육 카테고리에 해당하는 문서를 찾는 서브쿼리를 ...","[D29, D29, D15, D15, D28]","[D29, D29, D30, D30, D28]",0.4,0.0
23,Q24,0.651829,0.914790,카테고리가 환경인 문서 중 카테고리가 과학인 문서를 찾아줘,"""환경 카테고리에 속하는 문서에서 과학 카테고리에 해당하는 문서를 찾기 위한 서브쿼...","[D14, D14, D29, D29, D18]","[D30, D30, D29, D29, D28]",0.4,0.0
27,Q28,0.919684,0.628727,카테고리가 스포츠인 문서 중 카테고리가 역사인 문서를 찾아줘,"""스포츠 카테고리에 속하는 문서에서 역사 카테고리에 해당하는 문서를 찾는 서브쿼리를...","[D29, D29, D10, D10, D28]","[D29, D29, D30, D30, D28]",0.4,0.0
11,Q12,0.718924,0.924901,카테고리 건강에 속한 문서를 쓴 저자의 다른 문서를 찾아줘,"""건강 카테고리에 속한 문서를 쓴 저자의 이름을 기반으로 해당 저자가 쓴 다른 문서...","[D11, D11, D29, D29, D28]","[D29, D29, D11, D11, D28]",0.4,0.4


In [9]:
avg_hop1_time = results_df['hop1_time'].mean()
avg_hop2_time = results_df['hop2_time'].mean()
avg_p1 = results_df['P@5_hop1'].mean()
avg_p2 = results_df['P@5_hop2'].mean()

summary = pd.DataFrame({'단계': ['1-hop', '2-hop'],
                        '평균 지연(초)': [avg_hop1_time, avg_hop2_time],
                        '평균 P@5': [avg_p1, avg_p2]})
summary

,단계,평균 지연(초),평균 P@5
0,1-hop,0.751297,0.260000
1,2-hop,0.712547,0.153333
